In [8]:
import tensorflow as tf

In [9]:
tf.__version__

'2.0.0-beta1'

In [10]:
import os
import re
import h5py
import shutil
import skimage
import imageio
import numpy as np

from glob import glob
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

import matplotlib.pyplot as plt
%matplotlib inline

# don't print matching warnings
import warnings
warnings.filterwarnings('ignore') 

### Download dataset and unpack it

In [11]:
_URL = 'https://s3.amazonaws.com/nist-srd/SD18/sd18.zip'

path_to_zip = tf.keras.utils.get_file('sd18.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'sd18/')

### Organize all images

In [12]:
if not os.path.isdir('data'):
    os.mkdir('data')

filenames = glob(PATH + 'single/f1_p1/*/*.png')
for filename in tqdm(filenames):
    indx = filename.split('/')[-1].split('_')[0]
    # remove leading zeros from index
    indx = re.sub(r'(?<!\d)0+', '', indx)
    side = filename.split('/')[-1].split('_')[2].split('.')[0].lower()
    new_file = 'data/mugshot_{}.{}.png'.format(side, indx)
    shutil.copyfile(filename, new_file)

100%|██████████| 2434/2434 [00:09<00:00, 244.84it/s]


In [13]:
# Convert Grayscale to RGB 
# Resize to (256, 256)
filenames = glob('data/*.png')
for filename in tqdm(filenames):
    im = skimage.io.imread(filename)
    im = skimage.color.gray2rgb(im)
    im = skimage.transform.resize(im, (256, 256), anti_aliasing=True)
    im = skimage.util.img_as_ubyte(im)
    skimage.io.imsave(filename, im)

100%|██████████| 2434/2434 [05:41<00:00,  7.59it/s]


In [14]:
# Flip L to R
filenames = glob('data/mugshot_l.*.png')
for filename in tqdm(filenames):
    im = skimage.io.imread(filename)
    im = np.fliplr(im)
    skimage.io.imsave(filename, im)
    # rename file
    new_filename = filename.replace('_l', '_r')
    os.rename(filename, new_filename)

100%|██████████| 146/146 [00:03<00:00, 40.31it/s]


In [15]:
if not os.path.isdir('tmp'):
    os.mkdir('tmp')
    
if not os.path.isdir('data/test'):
    os.mkdir('data/test')

In [16]:
# train test split
frnt_files = sorted(glob('data/mugshot_f.*.png'))
side_files = sorted(glob('data/mugshot_r.*.png'))

mylist = list(zip(frnt_files, side_files))
for f in mylist[:int(len(mylist)*0.8)]:
    shutil.move(f[0], 'tmp')
    shutil.move(f[1], 'tmp')
    
for f in mylist[int(len(mylist)*0.8):]:
    shutil.move(f[0], 'data/test')
    shutil.move(f[1], 'data/test')

### Image-to-Image paper describes that it randomly jitter each image
1. resize image up
2. randomly crop back to org size
3. randomly flip horizontally

In [17]:
def load(image):
    image = tf.io.read_file(image)
    image = tf.image.decode_png(image)
    
    return tf.dtypes.cast(image, tf.float32)

In [18]:
@tf.function()
def random_jitter(image):
    # resizing to 286 x 286 x 3
    image = tf.image.resize(image, [286, 286], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # randomly cropping to 256 x 256 x 3
    image = tf.image.random_crop(image, size=[256, 256, 3])

    # randomly mirroring
    image = tf.image.random_flip_left_right(image)

    return image

In [19]:
# normalizing the images to [-1, 1]
def normalize(image):
    return (image / 127.5) - 1

In [20]:
if not os.path.isdir('data/train'):
    os.mkdir('data/train')

In [34]:
filenames = sorted(glob('tmp/mugshot_f.*.png'))
for filename in filenames:
    name = filename.split('/')[-1]
    image = load(filename)
    for i in range(4):
        image = random_jitter(image)
        image = normalize(image)
        nname = name.replace('_f', '_f' + str(i))
        imageio.imwrite(os.path.join('data/train/', nname), image)

W0710 15:48:49.525883 4792235456 util.py:64] Lossy conversion from float32 to uint8. Range [-0.686274528503418, 0.9215686321258545]. Convert image to uint8 prior to saving to suppress this warning.
W0710 15:48:49.624725 4792235456 util.py:64] Lossy conversion from float32 to uint8. Range [-1.0053825378417969, -0.9928335547447205]. Convert image to uint8 prior to saving to suppress this warning.
W0710 15:48:49.700994 4792235456 util.py:64] Lossy conversion from float32 to uint8. Range [-1.0078853368759155, -1.0077898502349854]. Convert image to uint8 prior to saving to suppress this warning.
W0710 15:48:49.764853 4792235456 util.py:64] Lossy conversion from float32 to uint8. Range [-1.0079050064086914, -1.007904291152954]. Convert image to uint8 prior to saving to suppress this warning.
W0710 15:48:49.791386 4792235456 util.py:64] Lossy conversion from float32 to uint8. Range [-0.5686274766921997, 0.8666666746139526]. Convert image to uint8 prior to saving to suppress this warning.
W071

In [35]:
filenames = sorted(glob('data/train/mugshot_f?.*.png'))
for k, filename in enumerate(filenames):
    shutil.move(filename, 'data/train/mugshot_front.' + str(k) + '.png') 

In [36]:
filenames = sorted(glob('tmp/mugshot_r.*.png'))
for filename in tqdm(filenames):
    for i in range(4):
        nname = filename.replace('_r', '_r' + str(i))
        shutil.copy2(filename, nname)

100%|██████████| 973/973 [00:02<00:00, 431.63it/s]


In [37]:
filenames = sorted(glob('tmp/mugshot_r?.*.png'))
for k, filename in enumerate(filenames):
    shutil.move(filename, 'data/train/mugshot_side.' + str(k) + '.png')

### Save the image dataset into a HDF5

In [38]:
hdf5_path = 'data/dataset.hdf5'

# train images
frnt_path = 'data/train/mugshot_front.*.png'
side_path = 'data/train/mugshot_side.*.png'

frnt = glob(frnt_path)
side = glob(side_path)

train_inpt = frnt[0:int(len(frnt))]
train_real = side[0:int(len(side))]

# test images
frnt_path = 'data/test/mugshot_f.*.png'
side_path = 'data/test/mugshot_r.*.png'

frnt = glob(frnt_path)
side = glob(side_path)

test_inpt = frnt[0:int(len(frnt))]
test_real = side[0:int(len(side))]

In [39]:
len(train_inpt)

3892

In [40]:
# Define an array for each of train and test set with the shape
# (number of data, image_height, image_width, image_depth)
train_shape = (len(train_inpt), 256, 256, 3)
test_shape = (len(test_inpt), 256, 256, 3)
    
# open a hdf5 file and create earrays
hdf5_file = h5py.File(hdf5_path, mode='w')

hdf5_file.create_dataset("train_inpt", train_shape, np.int8)
hdf5_file.create_dataset("train_real", train_shape, np.int8)
hdf5_file.create_dataset("test_inpt", test_shape, np.int8)
hdf5_file.create_dataset("test_real", test_shape, np.int8)

<HDF5 dataset "test_real": shape (244, 256, 256, 3), type "|i1">

In [41]:
for i in range(len(train_inpt)):
    img = skimage.io.imread(train_inpt[i])
    hdf5_file["train_inpt"][i, ...] = img[None]
    img = skimage.io.imread(train_real[i])
    hdf5_file["train_real"][i, ...] = img[None]

for i in range(len(test_inpt)):
    img = skimage.io.imread(test_inpt[i])
    hdf5_file["test_inpt"][i, ...] = img[None]
    img = skimage.io.imread(test_real[i])
    hdf5_file["test_real"][i, ...] = img[None]

In [42]:
hdf5_file.close()

### Check if the data is saved properly in the HDF5 file

In [43]:
# open the hdf5 file
hdf5_path = 'data/dataset.hdf5'
hdf5_file = h5py.File(hdf5_path, 'r')

# Get total number of samples
num_data = hdf5_file['train_inpt'].shape[0]
print(num_data)

3892


### Cleaning up behind me

In [44]:
# Removing all png images
files = glob('data/*.png')
for file in files:
    os.remove(file)
    
# remove tmp dir    
shutil.rmtree('tmp')

# Remove downloaded data
shutil.rmtree(PATH)